In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm 
import math

In [2]:
# parameter values
mu = 0.08
r = 0.04
sigma = 0.16
q = 0.02      #discount, hazard rate
K = 0.04      #fixed rate
sigma2 = sigma*sigma
num_years = 5
recovery = 0.4
# coupon values
###############################################################################
# statistical measure(real world)      
EpR_i = mu - 0.5 * sigma2 - K
Rp = EpR_i * np.ones(num_years)                        

# risk-neutral measure
EqR_i = r - 0.5 * sigma2 - K
Rq = EqR_i * np.ones(num_years)                          

# swap values
###############################################################################
# discount curve
Z = np.exp(-r * np.arange(0, num_years + 1))          #discount factor
PD = np.exp(-q * np.arange(0, num_years + 1))         

# value (the cash flows)   #r
discounted_cf = Z[1:] * Rq    
# values discounted to today                              
V = np.cumsum(discounted_cf[::-1])[::-1]         
V_0 = V[0]

# value of swap at each point in time [V(0), V(1), ..., V(num_years-1)]
V_t = V / Z[:-1]                                            

# expected exposures
###############################################################################
# expected exposure at end of period
EE_e = EpR_i + V_t[1:]  # mu
EE_s = V_t

In [3]:
### part 1 ###
notional_1 = 100 # notional 100m
#r:risk neutral for valuation.  swap value, CVA, DVA
exp_pos_v = (r - 0.5 * sigma2 - K) * norm.cdf((r - 0.5 * sigma2 - K) / (math.sqrt(5) * 0.2 * sigma)) + math.sqrt(5) * 0.2 * sigma * norm.pdf((r - 0.5 * sigma2 - K) / (math.sqrt(5) * 0.2 * sigma))
exp_neg_v = (r - 0.5 * sigma2 - K) * norm.cdf((-r + 0.5 * sigma2 + K) / (math.sqrt(5) * 0.2 * sigma)) - math.sqrt(5) * 0.2 * sigma * norm.pdf((-r + 0.5 * sigma2 + K) / (math.sqrt(5) * 0.2 * sigma))

#value
discounted_pos_v = Z[1:] * exp_pos_v
discounted_neg_v = Z[1:] * exp_neg_v
#values discounted to today
V_pos = np.cumsum(discounted_pos_v[::-1])[::-1]
V_neg = np.cumsum(discounted_neg_v[::-1])[::-1]
#value of swap at each point in time [V(0), V(1), ..., V(num_years-1)]
V_pos_t = V_pos / Z[:-1]
V_pos_t = np.append(V_pos_t, 0)
V_neg_t = V_neg / Z[:-1]
V_neg_t = np.append(V_neg_t, 0)
V_pos_t

array([0.10038913, 0.08188458, 0.06262485, 0.04257911, 0.02171529,
       0.        ])

In [4]:
## value of swap at time zero
V_0 = V_pos_t[0] * notional_1
V_0

10.03891333879865

In [5]:
#mu: real world measure. exposure
exp_pos_e = (mu - 0.5 * sigma2 - K) * norm.cdf((mu - 0.5 * sigma2 - K) / (math.sqrt(5) * 0.2 * sigma)) + (math.sqrt(5) * 0.2 * sigma) * norm.pdf((mu - 0.5 * sigma2 - K) / (math.sqrt(5) * 0.2 * sigma))
exp_neg_e = (mu - 0.5 * sigma2 - K) * norm.cdf((K - (mu - 0.5 * sigma2)) / (math.sqrt(5) * 0.2 * sigma)) - (math.sqrt(5) * 0.2 * sigma) *  norm.pdf((K - (mu - 0.5 * sigma2)) / (math.sqrt(5) * 0.2 * sigma))
#value
discounted_pos_e = Z[1:] * exp_pos_e
discounted_neg_e = Z[1:] * exp_neg_e
#values discounted to today
E_pos = np.cumsum(discounted_pos_e[::-1])[::-1]
E_neg = np.cumsum(discounted_neg_e[::-1])[::-1]
#value of swap at each point in time [V(0), V(1), ..., V(num_years-1)]
E_pos_t = E_pos / Z[:-1]
E_pos_t = np.append(E_pos_t, 0)
E_neg_t = E_neg / Z[:-1]
E_neg_t = np.append(E_neg_t, 0)

In [6]:
##EPE
EPE = np.average(E_pos_t[1:]) * notional_1
##ENE
ENE = np.average(E_neg_t[1:]) * notional_1
##EE
EE = (np.sum(E_pos_t[1:]) + np.sum(E_neg_t[1:])) / (num_years * 2) * notional_1
EE, EPE, ENE

(2.5128696766221656, 8.163861322537262, -3.13812196929293)

In [7]:
E_pos_t[1:]

array([0.16007713, 0.12242605, 0.0832384 , 0.04245147, 0.        ])

In [8]:
## potential future exposure with 95 percentile
PFE = np.percentile(E_pos_t[1:], 95) * notional_1
PFE

15.254691813569139

In [9]:
#CVA
CVA = np.sum(PD[:-1] * (1- np.exp(-q)) * (1 - recovery) * Z[1:] * V_pos_t[1:] * notional_1)
CVA

0.2246059783304838

In [10]:
#DVA
DVA = np.sum(PD[:-1] * (1- np.exp(-q)) * (1 - recovery) * Z[1:] * V_neg_t[1:] * notional_1)
DVA

-0.3518079587932898